In [59]:
from enum import Enum
from typing import List
import math

from enum import Enum

class EnvironmentType(Enum):
	OFFICES_AND_CLASSROOMS = 0
	BEDROOMS = 1
	KITCHENS_AND_DINING_ROOMS = 2
	BATHROOMS_AND_OTHER_FACILITIES = 3
	VARIOUS = 4
	SHOPS_AND_COMMERCIAL_ESTABLISHMENTS = 6
	RESTAURANTS_CAFETERIAS_BARS_AND_HOTELS = 7
	WAITING_ROOMS_AND_COMMUNAL_AREAS = 8
	MACHINE_ROOMS_AND_EQUIPMENT = 9
	BANKS_AND_LIBRARIES = 10
	CHURCHES_AND_TEMPLES = 11
	LABORATORIES = 12
	CORRIDORS_STAIRCASES_CIRCULATION_AREAS_AND_LOCKER_ROOMS = 13
	WAREHOUSES_DEPOSITS_AND_STORAGE_AREAS = 14
	LAUNDRIES_AND_WORKSHOPS = 15
	MUSEUMS_EXHIBITIONS_AND_ART_GALLERIES = 16
	AUDITORIUMS_CINEMAS_AND_THEATRES = 17
	HOSPITALS = 18
	GARAGES = 19
	GYMS_AND_SPORTS_FACILITIES = 20
	TRANSPORT_TERMINALS = 21

class Device:
	def __init__(self, name: str, power: float = None, power_factor: float = None, apparent_power: float = None) -> None:
		self.name = name
		self.power = self.get_power(power, power_factor, apparent_power)
		self.power_factor = self.get_power_factor(power, power_factor, apparent_power)
		self.apparent_power = self.get_apparent_power(power, power_factor, apparent_power)

		if not self.power:
			raise ValueError(f"Power not defined on {self.name}, two of the following must be defined: power, power_factor, apparent_power")
		
		if not self.power_factor:
			raise ValueError(f"Power factor not defined on {self.name}, two of the following must be defined: power, power_factor, apparent_power")
		
		if not self.apparent_power:
			raise ValueError(f"Apparent power not defined  on {self.name}, two of the following must be defined: power, power_factor, apparent_power")

	def get_apparent_power(self, power: float, power_factor: float, apparent_power: float) -> float:
		if apparent_power:
			return self.apparent_power
		
		if power and power_factor:
			return power / power_factor
		
		return None
	
	def get_power(self, power: float, power_factor: float, apparent_power: float) -> float:
		if power:
			return power
		
		if apparent_power and power_factor:
			return apparent_power * power_factor
		
		return None
	
	def get_power_factor(self, power: float, power_factor: float, apparent_power: float) -> float:
		if power_factor:
			return power_factor
		
		if power and apparent_power:
			return power / apparent_power
		
		return None
	
	def __str__(self) -> str:
		return f"{self.name} - {self.power} W - {self.power_factor} - {self.apparent_power} VA"

class LightType(Enum):
	INCANDESCENT = 0
	LED = 1
	COMPACT_FLUORESCENT = 2
	MIXED = 3
	LOW_PRESSURE_SODIUM = 4
	HIGH_PRESSURE_SODIUM = 5
	METAL_HALIDE = 6
	FLUORESCENT_WITH_STARTER_LOW_PF = 7
	QUICK_START_FLUORESCENT_LOW_PF = 8
	MERCURY_VAPOR_LOW_PF = 9
	FLUORESCENT_WITH_STARTER_HIGH_PF = 10
	QUICK_START_FLUORESCENT_HIGH_PF = 11
	MERCURY_VAPOR_HIGH_PF = 12

class Light(Device):
	def __init__(self, power: float, light_type: LightType, name: str = "Lampada") -> None:
		
		self.light_type = light_type

		super().__init__(name, power, self.get_power_factor_from_light_type())  

	def get_power_factor_from_light_type(self):
		return {
			LightType.INCANDESCENT: 1.0,
			LightType.LED: 0.65,  
			LightType.COMPACT_FLUORESCENT: 0.7,
			LightType.MIXED: 1.0,
			LightType.LOW_PRESSURE_SODIUM: 0.85,
			LightType.HIGH_PRESSURE_SODIUM: 0.4,
			LightType.METAL_HALIDE: 0.6,
			LightType.FLUORESCENT_WITH_STARTER_LOW_PF: 0.5,
			LightType.QUICK_START_FLUORESCENT_LOW_PF: 0.5,
			LightType.MERCURY_VAPOR_LOW_PF: 0.5,
			LightType.FLUORESCENT_WITH_STARTER_HIGH_PF: 0.85,
			LightType.QUICK_START_FLUORESCENT_HIGH_PF: 0.85,
			LightType.MERCURY_VAPOR_HIGH_PF: 0.85,
		}.get(self.light_type, 0.8)
	
	def __str__(self) -> str:
		return f"{self.name} - {round(self.power, 2)} W - PF: {round(self.power_factor, 2)} - {round(self.apparent_power, 2)} VA"


class Motor(Device):
	def __init__(self, power: float = None, power_factor: float = None, horse_power: float = None, efficiency: float = None,  name: str = "Motor") -> None:
		
		self.horse_power = self.get_horse_power(power, power_factor, horse_power, efficiency)

		if not self.horse_power:
			raise ValueError(f"Horse power not defined on {self.name}, two of the following must be defined: power, horse_power")

		power = self.horse_power * 746

		if not power_factor:
			power_factor = self.get_motor_power_factor(power, self.horse_power)

		super().__init__(name, power, power_factor)


	def get_horse_power(self, power: float, power_factor: float, horse_power: float, efficiency) -> float:
		if horse_power:
			return horse_power

		if power:
			power = power / 746
			return power + (power * (1 - efficiency))

		return None

	def get_motor_power_factor(self, power: float, horse_power: float) -> float:
		if (power < 600.0):
			return 0.5
		if (horse_power < 4.0):
			return 0.75
		if (horse_power < 50.0):
			return 0.85
		return 0.9


class AirConditioned(Device):
	def __init__(self, name: str = "Ar Condicionado", power: float = None, btu: float = None) -> None:
		if power and not btu:
			btu = power * 3412.14
		
		if btu and not power:
			power = btu / 3412.14
		
		if not power and not btu:
			raise ValueError(f"Power or BTU must be defined on {name}")

		self.btu = btu

		super().__init__(name, power, 0.8)


class Environment:
	def __init__(self, name: str, w: float, h: float, type: EnvironmentType, light: Light, specific_devices: List[ Device ]) -> None:
		self.name = name
		self.width = w
		self.height = h
		self.type = type
		self.light = light
		self.specific_devices = specific_devices

	@property
	def area(self):
		return self.width * self.height

	@property
	def perimeter(self):
		return 2 * (self.width + self.height)
	
	@property
	def tug_power(self):
		return self.minimum_tug_power()
	
	@property
	def tug_number(self):
		return self.minimum_tug_number()
	
	@property
	def tue_number(self):
		return self.minimum_tue_number()
	
	@property
	def tue_power(self):
		return sum([device.power for device in self.specific_devices])


	@property
	def light_density_per_square_meter(self):
		density = {
			EnvironmentType.OFFICES_AND_CLASSROOMS: 12,
			EnvironmentType.BEDROOMS: 4,
			EnvironmentType.KITCHENS_AND_DINING_ROOMS: 10,
			EnvironmentType.BATHROOMS_AND_OTHER_FACILITIES: 10,
			EnvironmentType.VARIOUS: 14,
			EnvironmentType.SHOPS_AND_COMMERCIAL_ESTABLISHMENTS: 15,
			EnvironmentType.RESTAURANTS_CAFETERIAS_BARS_AND_HOTELS: 12,
			EnvironmentType.WAITING_ROOMS_AND_COMMUNAL_AREAS: 8,
			EnvironmentType.MACHINE_ROOMS_AND_EQUIPMENT: 6,
			EnvironmentType.BANKS_AND_LIBRARIES: 15,
			EnvironmentType.CHURCHES_AND_TEMPLES: 14,
			EnvironmentType.LABORATORIES: 20,
			EnvironmentType.CORRIDORS_STAIRCASES_CIRCULATION_AREAS_AND_LOCKER_ROOMS: 8,
			EnvironmentType.WAREHOUSES_DEPOSITS_AND_STORAGE_AREAS: 10,
			EnvironmentType.LAUNDRIES_AND_WORKSHOPS: 6,
			EnvironmentType.MUSEUMS_EXHIBITIONS_AND_ART_GALLERIES: 13,
			EnvironmentType.AUDITORIUMS_CINEMAS_AND_THEATRES: 18,
			EnvironmentType.HOSPITALS: 16,
			EnvironmentType.GARAGES: 2,
			EnvironmentType.GYMS_AND_SPORTS_FACILITIES: 20,
			EnvironmentType.TRANSPORT_TERMINALS: 9
		}

		return density.get(self.type, 13)

	def minimum_light_points(self):
		n = math.floor((self.recommended_power() * math.exp(-0.09 * self.area)) / self.light.power)
		return n + 1 if n & 1 else n

	def recommended_power(self):
		return self.area * self.light_density_per_square_meter

	def minimum_light_power(self):
		if self.area < 6:
			return 100
		return 100 + (math.floor((self.area - 6) / 4) * 60)

	def light_power(self):
		return max(self.minimum_light_power(), self.recommended_power())

	def minimum_tug_number(self):
		if self.type in [EnvironmentType.KITCHENS_AND_DINING_ROOMS, EnvironmentType.LAUNDRIES_AND_WORKSHOPS]:
			return max(math.ceil(self.perimeter / 3.5), 2)
		elif self.type in [EnvironmentType.BATHROOMS_AND_OTHER_FACILITIES]:
			return max(math.ceil(self.perimeter / 6), 2)
		elif self.type in [EnvironmentType.VARIOUS, EnvironmentType.GARAGES, EnvironmentType.CORRIDORS_STAIRCASES_CIRCULATION_AREAS_AND_LOCKER_ROOMS]:
			return max(math.ceil(self.perimeter / 6), 2)
		elif self.type in [EnvironmentType.SHOPS_AND_COMMERCIAL_ESTABLISHMENTS]:
			if self.area <= 40:
				return max(math.ceil(self.perimeter / 3), math.ceil(self.area / 4))
			else:
				return 10 + math.ceil((self.area - 40) / 10)
		else:
			if self.area <= 6:
				return 2
			else:
				return 1 + math.ceil(self.perimeter / 5)

	def minimum_tug_power(self):
		if self.type in [EnvironmentType.KITCHENS_AND_DINING_ROOMS, EnvironmentType.BATHROOMS_AND_OTHER_FACILITIES, EnvironmentType.LAUNDRIES_AND_WORKSHOPS]:
			tug_number = self.minimum_tug_number()
			if tug_number <= 3:
				return 600 * tug_number
			else:
				return 1800 + ((tug_number - 3) * 100)
		elif self.type == EnvironmentType.SHOPS_AND_COMMERCIAL_ESTABLISHMENTS:
			return 200 * self.minimum_tug_number()
		else:
			return 100 * self.minimum_tug_number()

	def minimum_tue_number(self):
		return len(self.specific_devices)
	
		
	
	def __str__(self) -> str:
		description = f"""
Enviroment: 
	Name: {self.name} - Area: {round(self.area, 2)}m² - Perimeter: {round(self.perimeter, 2)}m
	Light: 
		Light: {self.light}
		Recommended Power: {round(self.recommended_power(), 2)}W
		Minimum Light Power: {round(self.minimum_light_power(), 2)}W
		Light Power: {round(self.light_power(), 2)}W
		Minimum Light Points: {round(self.minimum_light_points(), 2)}
	Tug: 
		Minimum Tug Number: {round(self.minimum_tug_number(), 2)}
		Minimum Tug Power: {round(self.minimum_tug_power(), 2)}W
	Tue: 
		Minimum Tue Number: {round(self.minimum_tue_number(), 2)}
"""
		return description.strip()
	

class Circuit():
	pass

class InstallationType(Enum):
	SINGLE_PHASE = 1,
	TWO_PHASE = 2,
	THREE_PHASE = 3,
	EXCLUSIVE_TRANSFORMER = 4

	def __gt__(self, other):
		return self.value > other.value
	
	def __lt__(self, other):
		return self.value < other.value

class Installation():

	def __init__(self, environments: List[Environment], installation_type: InstallationType = None) -> None:
		self.environments = environments
		if installation_type and installation_type > self.minimum_installation_type():
			self.installation_type = installation_type
		else:
			self.installation_type = self.minimum_installation_type()
	
	@property
	def light_power(self):
		return sum([environment.light_power() for environment in self.environments])
	
	@property
	def light_number(self):
		return sum([environment.minimum_light_points() for environment in self.environments])
	
	@property
	def tug_power(self):
		return sum([environment.minimum_tug_power() for environment in self.environments])
	
	@property
	def tug_number(self):
		return sum([environment.minimum_tug_number() for environment in self.environments])
	
	@property
	def tue_power(self):
		return sum([environment.tue_power for environment in self.environments])
	
	@property
	def tue_number(self):
		return sum([environment.tue_number for environment in self.environments])

	@property
	def total_demanded_power(self):
		return (self.light_power + self.tug_power) * self.demand_factor_for_lighting_tug() + self.tue_power * self.demand_factor_for_tue(self.tue_number)

	@property
	def total_demanded_apparent_power(self):
		return self.total_demanded_power / 0.95
	
	@property
	def total_demanded_current(self):
		return self.total_demanded_power / (220 * self.phases)

	@property
	def input_branch_current(self):
		return self.total_demanded_current * 1.25

	@property
	def phases(self):
		if self.installation_type == InstallationType.SINGLE_PHASE:
			return 1
		if self.installation_type == InstallationType.TWO_PHASE:
			return 2
		return 3


	def minimum_installation_type(self) -> InstallationType:
		if self.total_demanded_power <= 15000:
			return InstallationType.SINGLE_PHASE
		if self.total_demanded_power <= 25000:
			return InstallationType.TWO_PHASE
		if self.total_demanded_power <= 50000:
			return InstallationType.THREE_PHASE
		return InstallationType.EXCLUSIVE_TRANSFORMER

	def demand_factor_for_lighting_tug(self):
		power = self.light_power + self.tug_power

		if power <= 1000:
			return 0.86
		if power <= 2000:
			return 0.75
		if power <= 3000:
			return 0.66
		if power <= 4000:
			return 0.59
		if power <= 5000:
			return 0.52
		if power <= 6000:
			return 0.45
		if power <= 7000:
			return 0.40
		if power <= 8000:
			return 0.35
		if power <= 9000:
			return 0.31
		if power <= 10000:
			return 0.27
		return 0.24

	def demand_factor_for_tue(self, number_of_tue):

		demand_factor_map = {
			1: 1.00,
			2: 1.00,
			3: 0.84,
			4: 0.76,
			5: 0.70,
			6: 0.65,
			7: 0.60,
			8: 0.57,
			9: 0.54,
			10: 0.52,
			11: 0.49,
			12: 0.48,
			13: 0.46,
			14: 0.45,
			15: 0.44,
			16: 0.43,
			17: 0.42,
			18: 0.41,
			19: 0.40,
			20: 0.40,
			21: 0.39,
			22: 0.39,
			23: 0.39,
			24: 0.38,
			25: 0.38
		}

		return demand_factor_map.get(number_of_tue, 0.38)

	def __str__(self) -> str:
		description = f"""
Installation:
	Light Power: {round(self.light_power, 2)}W
	Light Number: {round(self.light_number, 2)}
	Tug Power: {round(self.tug_power, 2)}W
	Tug Number: {round(self.tug_number, 2)}
	Tue Power: {round(self.tue_power, 2)}W
	Tue Number: {round(self.tue_number, 2)}
	Total Demanded Power: {round(self.total_demanded_power, 2)}W
	Installation Type: {self.installation_type}
	Phases: {self.phases}
	Total Demanded Apparent Power: {round(self.total_demanded_apparent_power, 2)}VA
	Total Demanded Current: {round(self.total_demanded_current, 2)}A
	Input Branch Current: {round(self.input_branch_current, 2)}A
"""
		environment_description = "\n".join([str(environment) for environment in self.environments])
		return description.strip() + "\n" + environment_description.strip()



In [61]:
environments = [
	Environment(
		"Escritório 1", 
		2.30, 
		3.9, 
		EnvironmentType.OFFICES_AND_CLASSROOMS, 
		Light(15, LightType.LED), 
		[AirConditioned("Ar condicionado escritório 1", 2000)]
	),

	Environment(
		"Escritório 2", 
		2.70, 
		5.10, 
		EnvironmentType.OFFICES_AND_CLASSROOMS, 
		Light(15, LightType.LED), 
		[AirConditioned("Ar condicionado escritório 2", 2000)]
	),

	Environment(
		"Recepção", 
		2.80, 
		5.10, 
		EnvironmentType.WAITING_ROOMS_AND_COMMUNAL_AREAS, 
		Light(15, LightType.LED), 
		[AirConditioned("Ar condicionado recepção", 2000)]
	),

	Environment(
		"Vestiário", 
		2.25, 
		2.70, 
		EnvironmentType.BATHROOMS_AND_OTHER_FACILITIES, 
		Light(15, LightType.LED), 
		[]
	),

	Environment(
		"Cozinha", 
		2.70, 
		1.90, 
		EnvironmentType.KITCHENS_AND_DINING_ROOMS, 
		Light(15, LightType.LED), 
		[Device("Microondas", 2000, 0.92), Device("Chaleira", 1200, 1.0)]
	),

	Environment(
		"Refeitório", 
		2.70, 
		2.90, 
		EnvironmentType.RESTAURANTS_CAFETERIAS_BARS_AND_HOTELS, 
		Light(15, LightType.LED), 
		[]
	),

	Environment(
		"Depósito", 
		2.70, 
		1.30, 
		EnvironmentType.WAREHOUSES_DEPOSITS_AND_STORAGE_AREAS, 
		Light(15, LightType.LED), 
		[]
	),

	Environment(
		"Banheiro", 
		1.95, 
		1.2, 
		EnvironmentType.BATHROOMS_AND_OTHER_FACILITIES, 
		Light(15, LightType.LED), 
		[Device("Chapinha / Secador", 1200, 1.0)]
	),

	Environment(
		"Banho", 
		1.99562, 
		1.99562, 
		EnvironmentType.BATHROOMS_AND_OTHER_FACILITIES, 
		Light(15, LightType.LED), 
		[Device("Chuveiro", 7000, 1.0), Device("Chapinha / Secador", 1200, 1.0)]
	),

	Environment(
		"Produção 1", 
		7.5, 
		8, 
		EnvironmentType.LAUNDRIES_AND_WORKSHOPS, 
		Light(15, LightType.LED), 
		[Motor(horse_power = 5.0, efficiency = 0.92, name = "Batedor de argila")] + 
		6 * [Motor(name = "Molde e cortador", horse_power = 1.0, power_factor = 0.85, efficiency = 0.95)]
	),

	Environment(
		"Produção 2", 
		7.5, 
		8, 
		EnvironmentType.LAUNDRIES_AND_WORKSHOPS, 
		Light(15, LightType.LED), 
		2 * [Motor(horse_power = 2.0, power_factor = 0.8, efficiency = 0.95)] + 
		[Motor(horse_power = 2.0, power_factor = 0.83, efficiency = 0.93)] +
		[Device(name = "Forno", power = 6000, power_factor = 1.0)]
	),

	Environment(
		"Produção 3", 
		4, 
		10.4, 
		EnvironmentType.LAUNDRIES_AND_WORKSHOPS, 
		Light(15, LightType.LED), 
		2 * [Motor(horse_power = 1.0, power_factor = 0.85, efficiency = 0.95)]
	),

	Environment(
		"Empacotamento", 
		4, 
		10.4, 
		EnvironmentType.LAUNDRIES_AND_WORKSHOPS, 
		Light(15, LightType.LED), 
		2 * [Motor(horse_power = 1.0, power_factor = 0.75, efficiency = 0.90)]
	),

]

installation = Installation(environments, InstallationType.THREE_PHASE)

print(installation)


Installation:
	Light Power: 4012.88W
	Light Number: 24
	Tug Power: 17000W
	Tug Number: 65
	Tue Power: 40266.0W
	Tue Number: 23
	Total Demanded Power: 20746.83W
	Installation Type: InstallationType.THREE_PHASE
	Phases: 3
	Total Demanded Apparent Power: 21838.77VA
	Total Demanded Current: 31.43A
	Input Branch Current: 39.29A
Enviroment: 
	Name: Escritório 1 - Area: 8.97m² - Perimeter: 12.4m
	Light: 
		Light: Lampada - 15 W - PF: 0.65 - 23.08 VA
		Recommended Power: 107.64W
		Minimum Light Power: 100W
		Light Power: 107.64W
		Minimum Light Points: 4
	Tug: 
		Minimum Tug Number: 4
		Minimum Tug Power: 400W
	Tue: 
		Minimum Tue Number: 1
Enviroment: 
	Name: Escritório 2 - Area: 13.77m² - Perimeter: 15.6m
	Light: 
		Light: Lampada - 15 W - PF: 0.65 - 23.08 VA
		Recommended Power: 165.24W
		Minimum Light Power: 160W
		Light Power: 165.24W
		Minimum Light Points: 4
	Tug: 
		Minimum Tug Number: 5
		Minimum Tug Power: 500W
	Tue: 
		Minimum Tue Number: 1
Enviroment: 
	Name: Recepção - Area: 14.28